#Testing with input

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import re
import pickle
input_text = input("Paste the text")
input_text = f"""{text}"""
input_text = re.sub(r'[\'"‘’“”]', '', input_text)
input_text = re.sub('^.*\(Reuters\)\s*-\s*', '', input_text)

In [ ]:
input_text = [input_text]
cv_text = cv.transform(input_text)
tfidf_text = tfidf.transform(cv_text)

In [ ]:
def define_labels(prob):
  if prob >= 0.5:
    predicted_class = "Real"  # Real news
  else:
    predicted_class = "Fake"  # Fake news
  return predicted_class

###Transformers

In [ ]:
import shutil
import torch
# Copy the saved model folder or zip file from Google Drive to Colab
# If it's a folder:
shutil.copytree("/content/gdrive/MyDrive/425 models/transformer", "transformer")

'transformer'

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model
model = AutoModelForSequenceClassification.from_pretrained("/content/transformer")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/transformer")

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt")

# Forward pass through the model to obtain logits
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class label
predicted_label = torch.sigmoid(outputs.logits).squeeze().tolist()  # Assuming binary classification
if predicted_label >= 0.5:
    predicted_class = "Real news"
else:
    predicted_class = "Fake news"

predicted_score = torch.sigmoid(outputs.logits).item()  # Assuming binary classification

# Print the predicted class and probability score
print("Transformers predicted label:", predicted_class)
print("Transformers predicted score:", predicted_score)

Transformers predicted label: Fake news
Transformers predicted score: 0.499073326587677


### MLP

In [ ]:
#Load models in
with open('/content/gdrive/MyDrive/425 models/mlp.pkl', 'rb') as f:
    mlp_classifier = pickle.load(f)

with open('/content/gdrive/MyDrive/425 models/tfidfMLP.pkl', 'rb') as f:
    tfidf_vec = pickle.load(f)

In [ ]:
input_tfidf = tfidf_vec.transform(input_text)
mlp_prob = mlp_classifier.predict_proba(input_tfidf)
mlp_pred_label = define_labels(mlp_prob[0][1])
print("MLP predicted label: ", mlp_pred_label)
print("MLP predicted score: ", mlp_prob[0][1])

MLP predicted label:  Real
MLP predicted score:  0.7527478000548626


###SVM

In [ ]:
#Load models in
with open('/content/gdrive/MyDrive/425 models/svm.pkl', 'rb') as f:
    svm_classifier = pickle.load(f)

with open('/content/gdrive/MyDrive/425 models/vectorizer.pkl', 'rb') as f:
    cv = pickle.load(f)

with open('/content/gdrive/MyDrive/425 models/tfidf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

In [ ]:
svm_prob = svm_classifier.decision_function(tfidf_text)
svm_pred_label = define_labels(svm_prob)
print("SVM predicted label: ", svm_pred_label)
print("SVM predicted score: ", svm_prob[0] if svm_prob[0]>0 else 1+svm_prob[0])

SVM predicted label:  Fake
SVM predicted score:  0.22558431394877265


###Naive Bayes

In [ ]:
with open('/content/gdrive/MyDrive/425 models/naivebayes.pkl', 'rb') as f:
    nb_classifier = pickle.load(f)

In [ ]:
nb_prob = nb_classifier.predict_proba(tfidf_text)
nb_pred_label = define_labels(nb_prob[0][1])
print("Naive Bayes predicted label: ", nb_pred_label)
print("Naive Bayes predicted score: ", nb_prob[0][1])

Naive Bayes predicted label:  Fake
Naive Bayes predicted score:  0.2431653428091772


In [ ]:
model_list = ["Transformers", "MLP", "SVM", "Naive Bayes"]
label_list = [tfm_predicted_class, mlp_pred_label, svm_pred_label, nb_pred_label]
score_list = [tfm_predicted_score, mlp_prob[0][1], svm_prob[0] if svm_prob[0]>0 else 1+svm_prob[0], nb_prob[0][1]]
result_df = pd.DataFrame({"Model": model_list,
                        "Label": label_list,
                        "Score": score_list})
result_df

,Model,Label,Score
0,Transformers,Fake,0.499073
1,MLP,Real,0.752748
2,SVM,Fake,0.225584
3,Naive Bayes,Fake,0.243165
